# Tensor Structure

## [Python List] vs [Torch Tensor]

In [1]:
# Python List

a = [1.0, 2.0, 1.0]

print(a[0])
print(a[2])
print(a)

1.0
1.0
[1.0, 2.0, 1.0]


In [2]:
import torch

# Tensor

a = torch.ones(3)

print(a)
print(a[1])
print(float(a[1]))

print("-" * 20)

a[2] = 2.0
print(a)

print(a.dtype)

tensor([1., 1., 1.])
tensor(1.)
1.0
--------------------
tensor([1., 1., 2.])
torch.float32


In [3]:
ts = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

print(ts)
print(ts.shape)

print("-" * 20)

ts = torch.zeros(3, 2)
print(ts)

print("-" * 20)

ts = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(ts[0, 1])
print(ts[0])

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
torch.Size([3, 2])
--------------------
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
--------------------
tensor(1.)
tensor([4., 1.])


## BroadCasting Case 1) Not Using torch.einsum

In [4]:
img_t = torch.randn(3, 5, 5)
print(img_t)
print(img_t.shape)

tensor([[[ 1.5983, -0.3484,  0.5655,  0.3301,  1.1145],
         [-0.4947, -0.7328,  0.3209, -0.0710,  1.2660],
         [ 0.2566, -0.5516,  0.6858,  2.2369,  0.2389],
         [-0.3658, -1.5018,  0.0093,  1.0337,  0.9548],
         [-0.6740, -0.4026,  0.7824,  0.8234, -1.5986]],

        [[ 0.4735, -0.2425, -0.8112,  0.0550, -0.2995],
         [-0.4127, -0.4388, -1.1521, -0.8011,  0.3785],
         [-1.8889, -2.7838,  0.1517,  0.6229, -0.4998],
         [-0.0773,  1.6237,  1.0182, -0.3051,  1.2892],
         [-0.8027, -0.0293, -1.7940,  0.5115, -0.1719]],

        [[-0.7189,  0.9472,  0.8887,  0.1411, -0.3666],
         [-0.6374,  1.1175, -1.8483, -0.2069, -0.4688],
         [ 0.7709, -0.8909,  0.3729,  1.4639, -1.9464],
         [ 0.6226,  1.8907,  0.4442,  0.9469,  0.2237],
         [ 0.7057,  1.3127,  0.1154, -0.1406,  0.7674]]])
torch.Size([3, 5, 5])


In [5]:
weights = torch.tensor([0.2126, 0.7152, 0.0722])
weights

tensor([0.2126, 0.7152, 0.0722])

In [6]:
batch_t = torch.randn(2, 3, 5, 5)
batch_t

tensor([[[[-1.8390e+00, -6.1058e-02, -2.3992e+00,  3.8978e-01,  7.2499e-01],
          [ 1.6839e+00, -1.8949e+00,  3.6814e-01,  2.1233e-01, -2.1607e-01],
          [-1.0920e+00, -2.0470e-01, -2.7246e-01,  8.0659e-01,  7.5569e-01],
          [ 2.6040e-01,  1.5757e-01, -6.0189e-01, -3.4326e-01, -2.9322e-01],
          [ 9.5835e-01,  2.0278e-01,  9.0836e-02,  4.1205e-01,  8.7899e-01]],

         [[-7.9365e-01, -3.0581e-01, -3.8241e-01,  1.6052e+00,  4.6246e-01],
          [ 1.6527e+00,  5.4450e-02,  4.3255e-01,  1.3183e+00,  1.0210e-01],
          [ 4.9393e-01,  1.3192e-01,  1.3010e+00,  5.9772e-01,  2.2005e+00],
          [-1.2698e+00, -5.3810e-01,  5.0625e-01,  1.0178e+00,  1.3289e+00],
          [ 4.7395e-01,  8.1816e-01, -1.4656e+00, -1.7423e-01, -1.0069e+00]],

         [[ 1.3774e+00,  2.0508e+00,  1.5535e+00,  1.6857e-02, -4.6688e-01],
          [-1.4446e+00, -3.4252e-01, -5.9673e-01,  5.2440e-03,  2.6123e-01],
          [-8.9691e-01, -1.0433e+00,  2.3154e-01, -7.9168e-02,  1.0703e+

In [7]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)

print(img_gray_naive)
print(batch_gray_naive)

img_gray_naive.shape, batch_gray_naive.shape

tensor([[ 0.4510,  0.1187,  0.2143,  0.1754,  0.1495],
        [-0.5150, -0.0181, -0.8932, -0.3597,  0.3919],
        [-0.2871, -1.4088,  0.4035,  1.4412, -0.7358],
        [ 0.0598,  0.6709,  0.4906,  0.5585,  0.8225],
        [-0.2570,  0.2936, -0.2988,  0.3981, -0.3344]])
tensor([[[-0.4184,  0.5613, -0.4094,  0.6706,  0.2402],
         [ 0.6307, -0.7277,  0.0680,  0.5120,  0.0491],
         [-0.4983, -0.3720,  0.4200,  0.4417,  1.3422],
         [-0.6132, -0.3229,  0.3771,  0.7156,  0.0433],
         [ 0.5569,  1.0194, -0.7193,  0.0767,  0.0639]],

        [[ 0.6712,  0.6475,  1.5316, -0.6596,  1.2701],
         [-0.0750, -0.3405,  1.2510,  0.0341, -0.1881],
         [ 0.2346, -0.3705, -0.5854, -0.1371,  0.2597],
         [-0.5191,  0.9485,  0.5739,  0.0800,  0.2302],
         [ 0.2157,  0.0542, -0.5070,  0.0942,  0.1784]]])


(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [8]:
unsqeezed_weights = weights.unsqueeze(-1).unsqueeze(-1) # BroadCasting
print(f"{weights.shape} => {unsqeezed_weights.shape}") 
print(unsqeezed_weights)

img_weights = (img_t * unsqeezed_weights)
img_weights

torch.Size([3]) => torch.Size([3, 1, 1])
tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]])


tensor([[[ 3.3980e-01, -7.4079e-02,  1.2023e-01,  7.0175e-02,  2.3693e-01],
         [-1.0517e-01, -1.5580e-01,  6.8221e-02, -1.5103e-02,  2.6915e-01],
         [ 5.4563e-02, -1.1727e-01,  1.4580e-01,  4.7557e-01,  5.0788e-02],
         [-7.7764e-02, -3.1929e-01,  1.9753e-03,  2.1976e-01,  2.0298e-01],
         [-1.4329e-01, -8.5591e-02,  1.6633e-01,  1.7505e-01, -3.3986e-01]],

        [[ 3.3868e-01, -1.7347e-01, -5.8019e-01,  3.9338e-02, -2.1418e-01],
         [-2.9519e-01, -3.1386e-01, -8.2395e-01, -5.7294e-01,  2.7067e-01],
         [-1.3509e+00, -1.9910e+00,  1.0852e-01,  4.4552e-01, -3.5745e-01],
         [-5.5284e-02,  1.1613e+00,  7.2824e-01, -2.1824e-01,  9.2200e-01],
         [-5.7411e-01, -2.0955e-02, -1.2831e+00,  3.6583e-01, -1.2297e-01]],

        [[-5.1903e-02,  6.8390e-02,  6.4163e-02,  1.0187e-02, -2.6470e-02],
         [-4.6023e-02,  8.0686e-02, -1.3345e-01, -1.4938e-02, -3.3851e-02],
         [ 5.5659e-02, -6.4326e-02,  2.6920e-02,  1.0569e-01, -1.4053e-01],
        

In [9]:
batch_weights = (batch_t * unsqeezed_weights)
batch_weights

tensor([[[[-3.9097e-01, -1.2981e-02, -5.1006e-01,  8.2868e-02,  1.5413e-01],
          [ 3.5799e-01, -4.0285e-01,  7.8266e-02,  4.5140e-02, -4.5936e-02],
          [-2.3216e-01, -4.3519e-02, -5.7925e-02,  1.7148e-01,  1.6066e-01],
          [ 5.5362e-02,  3.3499e-02, -1.2796e-01, -7.2978e-02, -6.2339e-02],
          [ 2.0375e-01,  4.3110e-02,  1.9312e-02,  8.7603e-02,  1.8687e-01]],

         [[-5.6762e-01, -2.1872e-01, -2.7350e-01,  1.1480e+00,  3.3075e-01],
          [ 1.1820e+00,  3.8943e-02,  3.0936e-01,  9.4286e-01,  7.3024e-02],
          [ 3.5326e-01,  9.4350e-02,  9.3045e-01,  4.2749e-01,  1.5738e+00],
          [-9.0817e-01, -3.8485e-01,  3.6207e-01,  7.2792e-01,  9.5045e-01],
          [ 3.3897e-01,  5.8515e-01, -1.0482e+00, -1.2461e-01, -7.2011e-01]],

         [[ 9.9446e-02,  1.4806e-01,  1.1216e-01,  1.2171e-03, -3.3709e-02],
          [-1.0430e-01, -2.4730e-02, -4.3084e-02,  3.7862e-04,  1.8861e-02],
          [-6.4757e-02, -7.5325e-02,  1.6717e-02, -5.7160e-03,  7.7274e-

In [10]:
img_gray_weighted = img_weights.sum(-3)

print(img_weights.shape)
print(img_weights)

print("*" * 100)

print(img_gray_weighted.shape)
print(img_gray_weighted)

torch.Size([3, 5, 5])
tensor([[[ 3.3980e-01, -7.4079e-02,  1.2023e-01,  7.0175e-02,  2.3693e-01],
         [-1.0517e-01, -1.5580e-01,  6.8221e-02, -1.5103e-02,  2.6915e-01],
         [ 5.4563e-02, -1.1727e-01,  1.4580e-01,  4.7557e-01,  5.0788e-02],
         [-7.7764e-02, -3.1929e-01,  1.9753e-03,  2.1976e-01,  2.0298e-01],
         [-1.4329e-01, -8.5591e-02,  1.6633e-01,  1.7505e-01, -3.3986e-01]],

        [[ 3.3868e-01, -1.7347e-01, -5.8019e-01,  3.9338e-02, -2.1418e-01],
         [-2.9519e-01, -3.1386e-01, -8.2395e-01, -5.7294e-01,  2.7067e-01],
         [-1.3509e+00, -1.9910e+00,  1.0852e-01,  4.4552e-01, -3.5745e-01],
         [-5.5284e-02,  1.1613e+00,  7.2824e-01, -2.1824e-01,  9.2200e-01],
         [-5.7411e-01, -2.0955e-02, -1.2831e+00,  3.6583e-01, -1.2297e-01]],

        [[-5.1903e-02,  6.8390e-02,  6.4163e-02,  1.0187e-02, -2.6470e-02],
         [-4.6023e-02,  8.0686e-02, -1.3345e-01, -1.4938e-02, -3.3851e-02],
         [ 5.5659e-02, -6.4326e-02,  2.6920e-02,  1.0569e-01, 

In [11]:
batch_gray_weighted = batch_weights.sum(-3)

print(batch_weights.shape)
print(batch_weights)

print("*" * 100)

print(batch_gray_weighted.shape)
print(batch_gray_weighted)

torch.Size([2, 3, 5, 5])
tensor([[[[-3.9097e-01, -1.2981e-02, -5.1006e-01,  8.2868e-02,  1.5413e-01],
          [ 3.5799e-01, -4.0285e-01,  7.8266e-02,  4.5140e-02, -4.5936e-02],
          [-2.3216e-01, -4.3519e-02, -5.7925e-02,  1.7148e-01,  1.6066e-01],
          [ 5.5362e-02,  3.3499e-02, -1.2796e-01, -7.2978e-02, -6.2339e-02],
          [ 2.0375e-01,  4.3110e-02,  1.9312e-02,  8.7603e-02,  1.8687e-01]],

         [[-5.6762e-01, -2.1872e-01, -2.7350e-01,  1.1480e+00,  3.3075e-01],
          [ 1.1820e+00,  3.8943e-02,  3.0936e-01,  9.4286e-01,  7.3024e-02],
          [ 3.5326e-01,  9.4350e-02,  9.3045e-01,  4.2749e-01,  1.5738e+00],
          [-9.0817e-01, -3.8485e-01,  3.6207e-01,  7.2792e-01,  9.5045e-01],
          [ 3.3897e-01,  5.8515e-01, -1.0482e+00, -1.2461e-01, -7.2011e-01]],

         [[ 9.9446e-02,  1.4806e-01,  1.1216e-01,  1.2171e-03, -3.3709e-02],
          [-1.0430e-01, -2.4730e-02, -4.3084e-02,  3.7862e-04,  1.8861e-02],
          [-6.4757e-02, -7.5325e-02,  1.6717e-0

## Broadcasting Case 2) Using torch.einsum

In [12]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)

print(img_t.shape, weights.shape)
print(img_t)
print(weights)

img_gray_weighted_fancy == img_gray_weighted

torch.Size([3, 5, 5]) torch.Size([3])
tensor([[[ 1.5983, -0.3484,  0.5655,  0.3301,  1.1145],
         [-0.4947, -0.7328,  0.3209, -0.0710,  1.2660],
         [ 0.2566, -0.5516,  0.6858,  2.2369,  0.2389],
         [-0.3658, -1.5018,  0.0093,  1.0337,  0.9548],
         [-0.6740, -0.4026,  0.7824,  0.8234, -1.5986]],

        [[ 0.4735, -0.2425, -0.8112,  0.0550, -0.2995],
         [-0.4127, -0.4388, -1.1521, -0.8011,  0.3785],
         [-1.8889, -2.7838,  0.1517,  0.6229, -0.4998],
         [-0.0773,  1.6237,  1.0182, -0.3051,  1.2892],
         [-0.8027, -0.0293, -1.7940,  0.5115, -0.1719]],

        [[-0.7189,  0.9472,  0.8887,  0.1411, -0.3666],
         [-0.6374,  1.1175, -1.8483, -0.2069, -0.4688],
         [ 0.7709, -0.8909,  0.3729,  1.4639, -1.9464],
         [ 0.6226,  1.8907,  0.4442,  0.9469,  0.2237],
         [ 0.7057,  1.3127,  0.1154, -0.1406,  0.7674]]])
tensor([0.2126, 0.7152, 0.0722])


tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])

In [13]:
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)

print(batch_t.shape, weights.shape)
print(batch_t)
print(weights)

batch_gray_weighted_fancy == batch_gray_weighted

torch.Size([2, 3, 5, 5]) torch.Size([3])
tensor([[[[-1.8390e+00, -6.1058e-02, -2.3992e+00,  3.8978e-01,  7.2499e-01],
          [ 1.6839e+00, -1.8949e+00,  3.6814e-01,  2.1233e-01, -2.1607e-01],
          [-1.0920e+00, -2.0470e-01, -2.7246e-01,  8.0659e-01,  7.5569e-01],
          [ 2.6040e-01,  1.5757e-01, -6.0189e-01, -3.4326e-01, -2.9322e-01],
          [ 9.5835e-01,  2.0278e-01,  9.0836e-02,  4.1205e-01,  8.7899e-01]],

         [[-7.9365e-01, -3.0581e-01, -3.8241e-01,  1.6052e+00,  4.6246e-01],
          [ 1.6527e+00,  5.4450e-02,  4.3255e-01,  1.3183e+00,  1.0210e-01],
          [ 4.9393e-01,  1.3192e-01,  1.3010e+00,  5.9772e-01,  2.2005e+00],
          [-1.2698e+00, -5.3810e-01,  5.0625e-01,  1.0178e+00,  1.3289e+00],
          [ 4.7395e-01,  8.1816e-01, -1.4656e+00, -1.7423e-01, -1.0069e+00]],

         [[ 1.3774e+00,  2.0508e+00,  1.5535e+00,  1.6857e-02, -4.6688e-01],
          [-1.4446e+00, -3.4252e-01, -5.9673e-01,  5.2440e-03,  2.6123e-01],
          [-8.9691e-01, -1.0433

tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])

## Naming for Tensor

In [14]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels']) # Experimental(Not Stable)
weights_named

C:\Users\kck\AppData\Local\Temp\ipykernel_6808\3843218446.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1408.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels']) # Experimental(Not Stable)


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [15]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')

print(f"img named : {img_named.shape}, {img_named.names}")
print(f"batch named : {batch_named.shape}, {batch_named.names}")

img named : torch.Size([3, 5, 5]), ('channels', 'rows', 'columns')
batch named : torch.Size([2, 3, 5, 5]), (None, 'channels', 'rows', 'columns')


In [16]:
# The align_as() function fills in missing dimensions and converts existing dimensions to the correct order.
weights_aligned = weights_named.align_as(img_named)

weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [17]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [18]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

## Tensor's dtype

In [19]:
double_points = torch.ones(10, 2, dtype = torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype = torch.short)
print(double_points.dtype, short_points.dtype)

torch.float64 torch.int16


In [20]:
# Case 1) Converting data type
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

# Case 2) Converting data type
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype = torch.short)

In [21]:
points_64 = torch.rand(5, dtype = torch.double)
points_short = points_64.to(torch.short)

points_64 * points_short
# When inputs with multiple types are mixed together through operation, the largest type is automatically generated.

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## Tensor Storage Indexing Access

In [22]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [23]:
points_storage = points.storage()
print(points_storage[0])
print(points.storage()[1])

4.0
1.0


In [24]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
print(points)

points_storage[0] = 2.0
print(points)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])


## Tensor Internal Operation

In [25]:
a = torch.ones(3, 2)

'''
It ends with an underscore (_) like zero_(), 
but instead of a new tensor being passed as a result of the operation, 
the existing tensor is also converted.
'''

print(a.zero_())
print(a)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


## Tensor Meta Data (size, offset, stride)

In [26]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]

print(points.size(), points.shape)
print(points.storage_offset())
print(points.stride())

print("*" * 10)

print(second_point.size(), second_point.shape) # size() == shape()
print(second_point.storage_offset())
print(second_point.stride())

torch.Size([3, 2]) torch.Size([3, 2])
0
(2, 1)
**********
torch.Size([2]) torch.Size([2])
2
(1,)


In [27]:
second_point = points[1]    # Point the Same Tensor Storage
print(points)
second_point[0] = 10.0
print(points)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])


In [28]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()    # Clone New Tensor
print(points)
second_point[0] = 10.0
print(points)
print(second_point)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([10.,  3.])


## Tensor Transpose (Not Copy)

In [29]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [30]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

### Check about two tensor's id

In [31]:
id(points.storage()) == id(points_t.storage())

True

In [32]:
print(points.stride())
print(points_t.stride())

(2, 1)
(1, 2)


In [33]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
print(some_t.shape)
print(transpose_t.shape)
print(some_t.stride())
print(transpose_t.stride())

torch.Size([3, 4, 5])
torch.Size([5, 4, 3])
(20, 5, 1)
(1, 5, 20)


## Contiguous Tensor

In [34]:
points.is_contiguous()

True

In [35]:
points_t.is_contiguous()

False

### Make Contiguous Tensor

In [36]:
points = torch.tensor([[4.0 ,1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [37]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [38]:
points_t.stride()

(1, 2)

In [39]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [40]:
print(points_t_cont.stride())
print(points_t_cont.storage())

(3, 1)
 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]


In [41]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

## Tensor Storage Space [CPU <-> GPU]

In [42]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device = 'cuda')

In [43]:
points_gpu = points.to(device = 'cuda')

# if gpu counts are more than one
points_gpu = points.to(device = 'cuda:0')

points = 2 * points    # Operation in CPU
points_gpu = 2 * points.to(device = 'cuda')    # Operation in GPU

In [44]:
points_gpu = points_gpu + 4

In [45]:
points_cpu = points_gpu.to(device = 'cpu')

In [46]:
points_gpu = points.cuda()
points_gpu = points.cuda(0)

points_cpu = points_gpu.cpu()

## Numpy Compatible (Numpy <-> Tensor)

In [47]:
points = torch.ones(3, 4)

points_np = points.numpy()    # Numpy -> Tensor
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [48]:
points = torch.from_numpy(points_np)    # Tensor -> Numpy
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

## Tensor Serialization

### How to save when you plan to read only PyTorch

In [49]:
# Case 1) Save Tensor
torch.save(points, '.\\serialization_1.t')

In [50]:
# Case 2) Save Tensor

with open('.\\serialization_2.t', 'wb') as f:
    torch.save(points, f)

In [51]:
# Case 1) File Load

points = torch.load('.\\serialization_1.t')

In [52]:
# Case 2) File Load

with open('.\\serialization_2.t', 'rb') as f:
    points = torch.load(f)

### How to store tensor compatible when using multiple software -> Using h5py

In [53]:
import h5py

In [54]:
f = h5py.File('.\\h5py_tensor.hdf5', 'w')
dset = f.create_dataset('coords', data = points.numpy())
print(dset)
f.close()

<HDF5 dataset "coords": shape (3, 4), type "<f4">


In [55]:
f = h5py.File('.\\h5py_tensor.hdf5', 'r')
dset = f['coords']
last_points = dset[-2:]

In [56]:
last_points = torch.from_numpy(dset[-2:])
print(last_points, last_points.dtype)

f.close()

print(last_points)
print(dset)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]]) torch.float32
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])
<Closed HDF5 dataset>
